In [415]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import pandas as pd

## Convert strings to actual minutes
def convertmins(l, add1, add2):
    for w in l:
        x = int(w)
        if x < 100:
            if x <= 45:
                u = x
            else:
                u = x+add1
        else:
            y = int(w[2:])
            if x >= 450 and x<460:
                u = y + 45
            elif y >= 450 and y<460:
                u = y + 90 + add1
        l[l.index(w)] = str(u)
    if len(l) != 0:
        li = '*'.join(l)
    else:
        li = '-1'
    return li

datalist = []

for i in range(66342,66722):
    ## scrape HTML document from Premier League website
    url = 'https://www.premierleague.com/match/' + str(i)
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    ser=Service('/Users/josiahwu29/Downloads/chromedriver')
    driver = webdriver.Chrome(service=ser, options=options)
    driver.get(url)
    
    ## Scroll Down the webpage
    count = 0
    while True:
        driver.refresh()
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        Divs=driver.page_source
        if 'blogCard time-half majorEvent' in Divs:
            break
        
    ## Find Home & Away Teams, Matchweek
    hteam = driver.find_element(By.CSS_SELECTOR,'[class="team home"]').text
    ateam = driver.find_element(By.CSS_SELECTOR,'[class="team away"]').text
    Date = (driver.find_element(By.CSS_SELECTOR,'[class="matchDate renderMatchDateContainer"]').text)[4:]

    ## Find Additional Time
    fullt = driver.find_element(By.CSS_SELECTOR,'[class="blogCard time majorEvent"]').text
    halft = driver.find_element(By.CSS_SELECTOR,'[class="blogCard time-half majorEvent"]').text
    add_time1 = int(halft[halft.index('+')+1:halft.index("'")])
    add_time2 = int(fullt[fullt.index('+')+1:fullt.index("'")])
    totalmins = add_time1 + add_time2 + 90

    ## Find Goal Minutes
    g = driver.find_element(By.CSS_SELECTOR,'[class="matchEvents matchEventsContainer"]')
    hg = g.find_element(By.CSS_SELECTOR,'[class="home"]').text
    ag = g.find_element(By.CSS_SELECTOR,'[class="away"]').text
    hgoal = list(filter(None,(''.join([x for x in hg if x.isnumeric() or x == "\'"])).split("\'")))
    agoal = list(filter(None,(''.join([x for x in ag if x.isnumeric() or x == "\'"])).split("\'")))
    hgoals = convertmins(hgoal, add_time1, add_time2)
    agoals = convertmins(agoal, add_time1, add_time2)
    
    ## Add Goal Minutes to datalist
    Final_Data = []
    Final_Data.append(list(','.join([str(hteam),str(Date),hgoals,str(totalmins),str(i),'Home'])))
    print(','.join([str(hteam),str(Date),hgoals,str(totalmins),str(i),'Home']))
    Final_Data.append(list(','.join([str(ateam),str(Date),agoals,str(totalmins),str(i),'Away'])))
    print(','.join([str(ateam),str(Date),agoals,str(totalmins),str(i),'Away']))

df = pd.DataFrame(data=Final_Data, columns=['Team','Date','Goals','TotalMins','MatchID','HomeOrAway'])
df.to_csv('GoalMins.csv')

KeyboardInterrupt: 

In [1]:
## Find Number of goals scored
import pandas as pd
df2 = pd.read_csv('/Users/josiahwu29/Downloads/GoalMins.csv')
newl = []
l = list(df2['Goals'].fillna('-'))
for n in range(len(l)):
    y = l[n]
    if y == '-':
        newl.append(0)
    else:
        newl.append(str(y).count('*')+1)

new = pd.DataFrame(newl)
new.to_csv('GoalsNum.csv')